In [ ]:
#
# LLM with QnA and Google Search
#
#
!pip install youtube-transcript-api==0.5.0) # YouTube lib to get transcript
!pip install pytube==12.1.2                 # YouTube lib
!pip install openai==0.27.0                 # OpenAI GPT
!pip install transformers==4.26.1           
!pip install chromadb==0.3.21
!pip install python-dotenv==1.0.0
!pip install langchain==0.0.130
!pip install tiktoken==0.3.3
!pip install faiss-cpu==1.7.3

In [ ]:
from dotenv import dotenv_values
api_keys = dotenv_values('keys.txt')

import os
os.environ['OPENAI_API_KEY'] = api_keys['OPENAI_API_KEY']
os.environ['SERPER_API_KEY'] = api_keys['SERPER_API_KEY']

In [ ]:
import openai
from langchain.llms import OpenAI

llm = OpenAI()

In [23]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import YoutubeLoader

loader = YoutubeLoader("FtyKgybYkaE", True)
result = loader.load()

In [24]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
docs = text_splitter.split_documents(result)
len(docs)

11

In [25]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [26]:
from langchain.vectorstores import FAISS

vectorstore = FAISS.from_documents(docs, embeddings)

In [27]:
query = "When is the next Devoxx Belgium? Answer truthfully and when you don't know the answer say : 'I don't know'"

In [28]:
search_results = vectorstore.similarity_search(query)
search_results
len(search_results)

4

In [29]:
from langchain.chains.question_answering import load_qa_chain

chain = load_qa_chain(llm, chain_type="stuff")
chain.run(input_documents=search_results, question=query)

" I don't know."

In [30]:
#
# Let's use Google
#
from langchain.agents import load_tools
from langchain.agents import initialize_agent

tools = load_tools(["google-serper"], llm=llm)

In [31]:
query = "When is the next Devoxx Belgium?"

In [32]:
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
agent.run(query)



> Entering new AgentExecutor chain...
 I should look for when the next Devoxx Belgium event is taking place.
Action: Serper Search
Action Input: "Devoxx Belgium next event"
Observation: Devoxx Belgium 2022 takes place from October 10th until the 14th. Oct 10, 2022
Thought: This is the next upcoming event.
Final Answer: Devoxx Belgium 2022 takes place from October 10th until the 14th. Oct 10, 2022

> Finished chain.


'Devoxx Belgium 2022 takes place from October 10th until the 14th. Oct 10, 2022'

In [33]:
query = "When is Devoxx Belgium in 2023?"
agent.run(query)



> Entering new AgentExecutor chain...
 I should do a search for the event
Action: Serper Search
Action Input: "Devoxx Belgium 2023"
Observation: 20th edition : 2 - 6 October 2023. Devoxx Belgium. The 19th edition of Devoxx Belgium, a 5-days technology conference, saw over 3,200 attendees. CFP will open in June and run until mid July 2023. Registrations. UPDATE: We'll release two batches of tickets, one on August 16th at 9h00 AM CEST and another ... Devoxx Events. VoxxxedDays. Add all events to your calendar. Past Events. VOXXED. 16 Mar 2023. Voxxed Days Zurich 2023. More details. 2023 edition of Devoxx Belgium will be held at Kinepolis Antwerpen, Antwerp starting on 02nd October. It is a 5 day event organised by The Java Community ... Voxxed Days Brussels 2023 ... Voxxed Days, a spin-off of Devoxx, is coming to Brussels! Yes, you're reading it right, we're talking about a big in-person, first ... Voxxed Days, a spin-off of Devoxx, is coming to Brussels! Yes, you're reading it right, 

'Devoxx Belgium in 2023 will be held on October 2 - 6.'